<a href="https://colab.research.google.com/github/AlirezaGhavidel70/training-and-testing-deploying-the-AI-decision-agents/blob/main/Annual_Decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @author: Alireza Ghavidel
# This code shows the AI agent decisions for a specific year

# upload external file before run (Trained_model.pth)
from google.colab import files
files.upload()


Saving Trained_model.pth to Trained_model.pth


{'Trained_model.pth': b'PK\x03\x04\x00\x00\x08\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Y\x00\t\x00Memphis_PER_PDQN_30years_w1_5_w2_5_SemiRandom_FRGsystem_Map_Action_LHS_withoutDS/data.pklFB\x05\x00ZZZZZ\x80\x02ccollections\nOrderedDict\nq\x00)Rq\x01(X\r\x00\x00\x00layer1.weightq\x02ctorch._utils\n_rebuild_tensor_v2\nq\x03((X\x07\x00\x00\x00storageq\x04ctorch\nFloatStorage\nq\x05X\x01\x00\x00\x000q\x06X\x03\x00\x00\x00cpuq\x07K\xa8tq\x08QK\x00K\x18K\x07\x86q\tK\x07K\x01\x86q\n\x89h\x00)Rq\x0btq\x0cRq\rX\x0b\x00\x00\x00layer1.biasq\x0eh\x03((h\x04h\x05X\x01\x00\x00\x001q\x0fh\x07K\x18tq\x10QK\x00K\x18\x85q\x11K\x01\x85q\x12\x89h\x00)Rq\x13tq\x14Rq\x15X\r\x00\x00\x00layer2.weightq\x16h\x03((h\x04h\x05X\x01\x00\x00\x002q\x17h\x07M@\x02tq\x18QK\x00K\x18K\x18\x86q\x19K\x18K\x01\x86q\x1a\x89h\x00)Rq\x1btq\x1cRq\x1dX\x0b\x00\x00\x00layer2.biasq\x1eh\x03((h\x04h\x05X\x01\x00\x00\x003q\x1fh\x07K\x18tq QK\x00K\x18\x85q!K\x01\x85q"\x89h\x00)Rq#tq$Rq%X\r\x00\x00\x

In [ ]:
###############################################################################
############# User input values for testing/deployment the AI-agent ###########
###############################################################################

# Specify an individual bridge specifications that you want to test the trained agent ( it must be in the range of the trained model using LHS)
deck_area_test = 1009.02 # Bridge deck area, [square meter, m2]
L_d_test = 2 # Detour length, [kilometer]
ADT_test = 9540 # Average daily traffic, [vehicles per day]
r_Truck_test = 0.12 # Truck traffic ratio

# please input the current bridge condition ratings as follows: [Deck, Superstructure, Substructure]
CR = [8,5,8] # [CR_deck, CR_Superstructure, CR_Substructure], CR must be between 0-8 (0: worse condition, 8: perfect condition)

###############################################################################
###############################################################################
state = [x - 1 for x in CR]
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F



n_params = int(4)



# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


n_actions = int(64)
# Get the number of state observations
n_observations = len(state)
# DQN Agent
class DQN(nn.Module):

    def __init__(self, n_inputs, n_actions):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_inputs, 24)
        self.layer2 = nn.Linear(24, 24)
        self.layer3 = nn.Linear(24, n_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        return self.layer3(x)




n_observations = len(state)  # len(state) == 3
n_params = 4
n_inputs = n_observations + n_params  # 3 + 4 = 7

policy_net = DQN(n_inputs, n_actions)
policy_net.load_state_dict(torch.load('Trained_model.pth'))
policy_net.eval()


def select_action(inputs):
    sample = random.random()
    if sample == sample:
        with torch.no_grad():
            inputs_cpu = inputs.to('cpu').numpy()

            policy_type = 0

            if policy_type == 0:
                action_values = policy_net(inputs)
                max_action = action_values.max(1)[1]  # Get the index of the maximum action

                # Convert max_action to a numpy array and then unravel it
                max_action_numpy = max_action.cpu().detach().numpy()
                modified_action = np.asarray(np.unravel_index(max_action_numpy, (4, 4, 4)))
                # here we don't let to take any action when the component is in the best condition
                if inputs_cpu[0][0] >= 6 :
                    modified_action[0] = 0

                if inputs_cpu[0][1] >= 6 :
                    modified_action[1] = 0

                if inputs_cpu[0][2] >= 6 :
                    modified_action[2] = 0



            # uppdate the suggested action based on the correlation of components
            if modified_action[2] == 3:
                modified_action[0] = 3
                modified_action[1] = 3

            if modified_action[1] == 3:
                modified_action[0] = 3

            return torch.tensor([np.asarray(np.ravel_multi_index(modified_action, (4, 4, 4)))], device=device, dtype=torch.long), action_values


""" Test """


n_params = 4  # Number of bridge parameters
state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

deck_area = deck_area_test
L_d = L_d_test
ADT = ADT_test
r_Truck = r_Truck_test

deck_area = torch.tensor([deck_area], dtype=torch.float32, device = device).unsqueeze(0)
L_d = torch.tensor([L_d], dtype=torch.float32, device = device).unsqueeze(0)
ADT = torch.tensor([ADT], dtype=torch.float32, device = device).unsqueeze(0)
r_Truck = torch.tensor([r_Truck], dtype=torch.float32, device = device).unsqueeze(0)
device = "cpu"
input_temp =  torch.hstack((state,deck_area,L_d,ADT,r_Truck))
input_temp = torch.tensor(input_temp, dtype=torch.float32, device = device)
action_DRL, Q_values = select_action(input_temp)

actions = np.unravel_index(action_DRL, (4, 4, 4))
print("**************************** Suggested actions by AI ****************************")
if actions[0]==0:
  print("Do nothing for Deck")
if actions[0]==1:
  print("Minor maintenance for Deck")
if actions[0]==2:
  print("Major maintenance for Deck")
if actions[0]==3:
  print("Replacement for Deck")

if actions[1]==0:
  print("Do nothing for Superstructure")
if actions[1]==1:
  print("Minor maintenance for Superstructure")
if actions[1]==2:
  print("Major maintenance for Superstructure")
if actions[1]==3:
  print("Replacement for Superstructure")

if actions[2]==0:
  print("Do nothing for Substructure")
if actions[2]==1:
  print("Minor maintenance for Substructure")
if actions[2]==2:
  print("Major maintenance for Substructure")
if actions[2]==3:
  print("Replacement for Substructure")

print("*********************************************************************************")



**************************** Suggested actions by AI ****************************
Do nothing for Deck
Minor maintenance for Superstructure
Do nothing for Substructure
*********************************************************************************


<ipython-input-94-f7d9ce321fd8>:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_temp = torch.tensor(input_temp, dtype=torch.float32, device = device)
